# <center><font color='darkblue'>Projeto 1 - Sistemas de Recomendação de filmes</font></center>
### <font color='darkblue'>Vetores e Espaço Vetorial em Sistemas de Recomendação</font>
### <font color='darkblue'>Matemática e Estatística Aplicada Para Data Science, Machine Learning e IA</font>
### <font color='royalblue'>Beartiz Andrade</font>

Nesse projeto vamos construir um sistema de recomendação de filmes completo.

Cada vez que um usuário assistir a um filme o sistema deve recomendar cinco outros filmes a fim de manter o usuário conectado na tela tanto quanto possível. Criarei então um sistema de recomendação via filtragem baseada em conteúdo. Para isso  usaremos  dados  reais  disponíveis  publicamente.

Os  dados  foram gerados  a  partir  do  site: https://developer.themoviedb.org/docsOs

## Instalando e carregando pacotes

In [1]:
#!pip install -q -U watermark

In [2]:
#!pip install pipreqs

In [3]:
#imports
import ast #processamento de texto
import nltk #processamento de texto
import sklearn #vetorização
import numpy as np #calculos
import pandas as pd #planilhas
from nltk.stem.porter import PorterStemmer #pre-processamento
from sklearn.feature_extraction.text import CountVectorizer #Transforma texto em números (vetores) contando a frequência de palavras
from sklearn.metrics.pairwise import cosine_similarity #calcula semalhança
pd.options.mode.chained_assignment = None #desativa aviso (warning) do pandas

In [4]:
#versões dos pcts usanos no JN
%reload_ext watermark
%watermark -a "Beatriz Andrade"

Author: Beatriz Andrade



## Carregando e compreendendo dados

In [5]:
#baixando datasets e informações
df_ba_filmes = pd.read_csv("dados/dataset_filmes.csv")
df_ba_filmes.shape

(4803, 20)

In [6]:
df_ba_filmes.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [7]:
df_elenco = pd.read_csv("dados/dataset_elenco.csv")
df_elenco.shape

(4803, 4)

In [8]:
df_elenco.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Organização dos Dados de Texto

> Existe uma relação entre os 2 datasets. Vamos fazer o merge pela coluna comum entre eles, a coluna 'title'.

In [9]:
#merge dos dataframes
df_ba_filmes = df_ba_filmes.merge(df_elenco, on = 'title') #concatenar pela coluna título
df_ba_filmes.shape

(4809, 23)

In [10]:
df_ba_filmes.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [11]:
df_ba_filmes.info() #determinar os tipos (float64, int64, object…)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [12]:
#filtrando o df e mantendo as colunas mais “importantes” (eu qm determino quais colunas)
df_ba_filmes = df_ba_filmes[['movie_id', 'title', 'overview', 'genres', 'cast', 'crew', 'keywords']]

In [13]:
df_ba_filmes.head()

,movie_id,title,overview,genres,cast,crew,keywords
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."


In [14]:
df_ba_filmes.shape

(4809, 7)

## Análise Exploratória e Limpeza dos Dados

In [15]:
df_ba_filmes.isnull().sum() #checando valores ausentes

movie_id    0
title       0
overview    3
genres      0
cast        0
crew        0
keywords    0
dtype: int64

In [16]:
df_ba_filmes.dropna(inplace = True) #remover linhas com vlr ausentes e salvar no df existente
df_ba_filmes.isnull().sum() #confirmando se não restou nada com VA

movie_id    0
title       0
overview    0
genres      0
cast        0
crew        0
keywords    0
dtype: int64

In [17]:
df_ba_filmes.duplicated().sum() #checando duplicados

np.int64(0)

In [18]:
df_ba_filmes.shape

(4806, 7)

In [19]:
df_ba_filmes.head()

,movie_id,title,overview,genres,cast,crew,keywords
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."


## Processamento de Texto com Abstract Syntax Trees

Vamos começar ajustando a coluna de gênero de filme. Nesta coluna precisamos apenas dos nomes e não dos ids.

In [20]:
# ast ajuda a descobrir programaticamente como é a gramática atual de um objeto processando árvores da gramática de sintaxe abstrata
ast.literal_eval('[{"id": 28, "name": "Action"},\
	              {"id": 12, "name": "Adventure"},\
	              {"id": 14, "name": "Fantasy"},\
	              {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [21]:
#Loop sobre os elementos da estrutura de dados. Usamos ast.literal_eval(obj) para avaliar/converter a string obj em uma estrutura de dados Pyth on
def converter(obj): 
    L = [] #lista vazia
    for i in ast.literal_eval(obj):
        L.append(i['name']) #se ao invés de 'name' fosse 'id' ele traria ['28', '12', '14', '878']
    return L

In [22]:
#testar a função
teste = converter('[{"id": 28, "name": "Action"},\
                   {"id": 12, "name": "Adventure"},\
                   {"id": 14, "name": "Fantasy"},\
                   {"id": 878, "name": "Science Fiction"}]')
print(teste)

['Action', 'Adventure', 'Fantasy', 'Science Fiction']


In [23]:
#aplicando a função à coluna 'genres' e salvando na coluna 'genres'
df_ba_filmes['genres'] = df_ba_filmes['genres'].apply(converter)
df_ba_filmes.head()

,movie_id,title,overview,genres,cast,crew,keywords
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."


In [24]:
#aplicando a função à coluna 'keywords' e salvando na coluna 'keywords'
df_ba_filmes['keywords'] = df_ba_filmes['keywords'].apply(converter)
df_ba_filmes.head()

,movie_id,title,overview,genres,cast,crew,keywords
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[culture clash, future, space war, space colon..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[ocean, drug abuse, exotic island, east india ..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[spy, based on novel, secret agent, sequel, mi..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[dc comics, crime fighter, terrorist, secret i..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[based on novel, mars, medallion, space travel..."


In [25]:
#aplicar ast na coluna cast e extrair sobre 3 membros do elenco
def converter3(obj): 
    L = []
    counter = 0 #inicializa contador para controlar o número de elementos adicionados á lista
    for i in ast.literal_eval(obj):
        if counter != 3: #verif. se o contador é diferente de 3
            L.append(i['name']) #adiciona o valor da chave ‘name’ do dic. atual à lista L.
            counter += 1 #incrementa o contador
        else:
            break #interrompe o contador se chegar a 3
    return L

In [26]:
#aplicando a função à coluna 'cast' e salvando em 'cast'
df_ba_filmes['cast'] = df_ba_filmes['cast'].apply(converter3)
df_ba_filmes.tail() #final

,movie_id,title,overview,genres,cast,crew,keywords
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[Action, Crime, Thriller]","[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...","[united states–mexico barrier, legs, arms, pap..."
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[Comedy, Romance]","[Edward Burns, Kerry Bishé, Marsha Dietlein]","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",[]
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[Comedy, Drama, Romance, TV Movie]","[Eric Mabius, Kristin Booth, Crystal Lowe]","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...","[date, love at first sight, narration, investi..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],"[Daniel Henney, Eliza Coupe, Bill Paxton]","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",[]
4808,25975,My Date with Drew,Ever since the second grade when he first saw ...,[Documentary],"[Drew Barrymore, Brian Herzlinger, Corey Feldman]","[{""credit_id"": ""58ce021b9251415a390165d9"", ""de...","[obsession, camcorder, crush, dream girl]"


In [27]:
#extrair nome do diretor do filme
def fetch_director(obj): 
    L = []
    for i in ast.literal_eval(obj): #converte a string 'obj' em um objeto python e itera sobre seus elementos.
        if i['job'] == 'Director': #verif. se o trabalho ('job') do elemento atual é 'Director'.
            L.append(i['name']) #adiciona o valor da chave ‘name’ à lista L.
            break #interrompe o loop após o primeiro diretor
    return L

In [28]:
#aplicando a função à coluna 'crew'
df_ba_filmes['crew'] = df_ba_filmes['crew'].apply(fetch_director)
df_ba_filmes.head()

,movie_id,title,overview,genres,cast,crew,keywords
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[culture clash, future, space war, space colon..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],"[ocean, drug abuse, exotic island, east india ..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],"[spy, based on novel, secret agent, sequel, mi..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],"[dc comics, crime fighter, terrorist, secret i..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],"[based on novel, mars, medallion, space travel..."


In [29]:
#separando a string da coluna overview por espaço em branco (separar palavras por vírgulas)
df_ba_filmes['overview'] = df_ba_filmes['overview'].apply(lambda x:x.split())
df_ba_filmes.head()

,movie_id,title,overview,genres,cast,crew,keywords
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[culture clash, future, space war, space colon..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],"[ocean, drug abuse, exotic island, east india ..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],"[spy, based on novel, secret agent, sequel, mi..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],"[dc comics, crime fighter, terrorist, secret i..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],"[based on novel, mars, medallion, space travel..."


In [30]:
#replace de espaço por vazio (remove o espaço entre palavras ex: science fiction para sciencefiction)
df_ba_filmes['genres'] = df_ba_filmes['genres'].apply(lambda x:[i.replace(" ","") for i in x])
df_ba_filmes['keywords'] = df_ba_filmes['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
df_ba_filmes['cast'] = df_ba_filmes['cast'].apply(lambda x:[i.replace(" ","") for i in x])
df_ba_filmes['crew'] = df_ba_filmes['crew'].apply(lambda x:[i.replace(" ","") for i in x])
df_ba_filmes.head()

,movie_id,title,overview,genres,cast,crew,keywords
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[cultureclash, future, spacewar, spacecolony, ..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[ocean, drugabuse, exoticisland, eastindiatrad..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[spy, basedonnovel, secretagent, sequel, mi6, ..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[dccomics, crimefighter, terrorist, secretiden..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[basedonnovel, mars, medallion, spacetravel, p..."


In [31]:
#criando a coluna de ‘tags’, nesse caso um vetor de strings com os valores das colunas
df_ba_filmes['tags'] = df_ba_filmes['overview'] + \
                       df_ba_filmes['genres'] + \
                       df_ba_filmes['keywords'] + \
                       df_ba_filmes['cast'] + \
                       df_ba_filmes['crew']


In [32]:
df_ba_filmes.head()

,movie_id,title,overview,genres,cast,crew,keywords,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili..."


In [33]:
#dataset final
df_ba_filmes_novo = df_ba_filmes[['movie_id', 'title', 'tags']]
df_ba_filmes_novo.head(10)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
5,559,Spider-Man 3,"[The, seemingly, invincible, Spider-Man, goes,..."
6,38757,Tangled,"[When, the, kingdom's, most, wanted-and, most,..."
7,99861,Avengers: Age of Ultron,"[When, Tony, Stark, tries, to, jumpstart, a, d..."
8,767,Harry Potter and the Half-Blood Prince,"[As, Harry, begins, his, sixth, year, at, Hogw..."
9,209112,Batman v Superman: Dawn of Justice,"[Fearing, the, actions, of, a, god-like, Super..."


In [34]:
#join da string simplificar o vetor, juntar tudo em uma string
df_ba_filmes_novo['tags'] = df_ba_filmes_novo['tags'].apply(lambda x:" ".join(x)) 

#colocar em minúsculo para evitar a diferença de palavras maiúsculas e minusculas
df_ba_filmes_novo['tags'] = df_ba_filmes_novo['tags'].apply(lambda x:x.lower()) 

## Parse e Vetorização

> Stemming: é o processo de redução de uma palavra ao seu radical que está ligado a sufixos e prefixos ou às raízes de palavras conhecidas como "lemmas". Stemming é importante na compreensão de linguagem natural e processamento de linguagem natural.

In [35]:
#criar o parser para ler e interpretar dados e transformá-los em algo utilizável (organizado)
parser_ps = PorterStemmer() 

In [36]:
def stem(text): #função stemming
    y = []
    for i in text.split(): #divide a string em palavras e itera sobre elas
        y.append(parser_ps.stem(i)) #faz o stemming na palavra atual i e adiciona o resultado na lista y
    return " ".join(y) #retorna palavras processadas como string, unindo-as com espaços

In [37]:
#aplica a função a coluna tags
df_ba_filmes_novo['tags'] = df_ba_filmes_novo['tags'].apply(stem) 
df_ba_filmes_novo.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


In [38]:
#criar vetor com no max 5000 atributos
cv = CountVectorizer(max_features = 5000, stop_words = 'english') 

A vetorização no contexto de processamento de linguagem natural (PLN) é o processo de converter texto em uma representação numérica, geralmente na forma de vetores. Este processo é fundamental para que algoritmos de aprendizado de máquina possam trabalhar com dados textuais, uma vez que eles requerem entradas numéricas.

A linha de código acima está criando uma instância da classe CountVectorizer do scikit-learn. Vamos detalhar o que acontece nesta linha:

**CountVectorizer**: É uma técnica de vetorização que converte uma coleção de documentos de texto em uma matriz de contagens de tokens. Basicamente, ela conta quantas vezes cada palavra ocorre em cada documento.

**max_features = 5000**: Este parâmetro limita o número de palavras mais frequentes a serem consideradas. Aqui, apenas as 5000 palavras mais frequentes em todo o conjunto de documentos serão mantidas.

**stop_words = 'english'**: Especifica que as palavras de parada (stop words) em inglês devem ser removidas do texto. Palavras de parada são palavras que são filtradas antes ou após o processamento de texto, geralmente porque são muito comuns e carregam pouca informação significativa (por exemplo, "the", "is", "and" em inglês).

In [39]:
#cria vetores para tags
#analisa a coluna e identifica tokens (palavras únicas) e converte em matriz de vetores
#a saída do fit_transform é geralmente uma matriz esparsa (sparse matrix), que economiza memória ao armazenar apenas os valores não nulos.
#o método .toarray() converte essa matriz esparsa em uma matriz NumPy densa (um numpy.ndarray bidimensional), 
#onde todos os valores (incluindo os zeros) são explicitamente armazenados.
vectors = cv.fit_transform(df_ba_filmes_novo['tags']).toarray() 

In [40]:
len(cv.get_feature_names_out()) #inspecionar o tamanho do vocabulário

5000

In [41]:
type(vectors)

numpy.ndarray

In [42]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4806, 5000))

In [43]:
#por padrão, o NumPy esconde parte dos dados quando o array é muito grande)
np.set_printoptions(threshold = np.inf) #visualizar as colunas do array

> Cada vetor é um filme e ele verifica se há a palavra em algum lugar na linha do outro filme

A matriz **vectors** gerada pela função fit_transform do CountVectorizer representa a frequência de cada palavra (após a aplicação de stemming) em cada documento (ou linha) da coluna de tags de seu dataframe. Aqui está o que significa cada elemento dessa matriz:

0: Indica que a palavra correspondente a essa posição na matriz não aparece na linha específica do dataframe. Ou seja, a tag após o stemming não está presente naquele registro específico.

1: Significa que a palavra aparece uma vez na linha do dataframe. Após o stemming, a tag foi identificada uma vez naquele registro.

2 (ou mais): Indica que a palavra aparece múltiplas vezes. Um valor de 2 significa que, após o stemming, a tag específica foi encontrada duas vezes naquele registro, e assim por diante para valores maiores.

Este comportamento é esperado porque o CountVectorizer constrói um vocabulário de todas as palavras únicas encontradas nos documentos fornecidos e, em seguida, conta quantas vezes cada palavra aparece em cada documento. O processo de stemming aplicado anteriormente por meio da função stem reduz as palavras a suas raízes, o que pode resultar na agregação de diferentes formas da mesma palavra como uma única entrada no vocabulário. Isso pode levar a um aumento na contagem (por exemplo, "run", "running", "runs" após stemming podem ser contados como a mesma palavra base, aumentando sua contagem em um documento específico).

In [44]:
vectors[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [45]:
vectors[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Cálculo de Distância dos Vetores

A célula abaixo trabalha com os vetores no espaço de dimensão para calcular a distância matemática entre eles usando a distância de cosseno (cosine_similarity).

In [46]:
#calcula as similaridades entre os vetores calculando as distâncias entre eles
similaridades = cosine_similarity(vectors) 

## Construindo o Sistema de Recomendação

In [47]:
#função do sistema de recomendação
def sistema_recomendacao(movie): 
    index = df_ba_filmes_novo[df_ba_filmes_novo['title'] == movie].index[0] #obtem o indice do filme 
        #passando como argumento o filme que o usuário assistiu
    distances = sorted(list(enumerate(similaridades[index])), reverse = True, key = lambda x: x[1]) #verificamos os filmes
        #com vetores de menos distância para o filme argumento
    for i in distances[1:6]: #considera os 5 filmes com vetor de menor distancia, ou seja, maior similaridade
        print(df_ba_filmes_novo.iloc[i[0]].title)

## Aplicando o Sistema de Recomendação

In [48]:
sistema_recomendacao('Avengers: Age of Ultron')

Iron Man 3
Iron Man 2
Iron Man
Thor
The Avengers


In [49]:
sistema_recomendacao('Jurassic World')

Jurassic Park
The Lost World: Jurassic Park
Walking With Dinosaurs
Terminator Genisys
Jurassic Park III


In [50]:
sistema_recomendacao('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [51]:
%reload_ext watermark
%watermark -a "Beatriz Andrade"

Author: Beatriz Andrade



In [52]:
%watermark -v -m

Python implementation: CPython
Python version       : 3.13.5
IPython version      : 8.30.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : AMD64 Family 25 Model 80 Stepping 0, AuthenticAMD
CPU cores   : 16
Architecture: 64bit



In [53]:
%watermark --iversions

nltk   : 3.9.1
numpy  : 2.3.1
sklearn: 1.7.1
pandas : 2.3.0



In [54]:
!pip freeze > requirements.txt
print("✅ Arquivo requirements.txt criado com pip freeze")

✅ Arquivo requirements.txt criado com pip freeze


In [55]:
# Ver o conteúdo:
with open('requirements.txt', 'r') as f:
    print(f.read())

accelerate==0.26.0
aiobotocore @ file:///C:/b/abs_431uzunk2v/croot/aiobotocore_1738237882529/work
aiohappyeyeballs==2.6.1
aiohttp==3.12.13
aioitertools @ file:///tmp/build/80754af9/aioitertools_1607109665762/work
aiosignal==1.3.2
alabaster @ file:///C:/Users/dev-admin/perseverance-python-buildout/croot/alabaster_1729041938345/work
altair @ file:///C:/b/abs_0b1avjyzoc/croot/altair_1743016738596/work
anaconda-anon-usage @ file:///C:/b/abs_1az9wuczyu/croot/anaconda-anon-usage_1749054795139/work
anaconda-auth @ file:///C:/b/abs_66s84uss7v/croot/anaconda-cloud-auth-split_1747863792543/work
anaconda-catalogs @ file:///C:/b/abs_ac59ma63va/croot/anaconda-catalogs_1747774731997/work
anaconda-cli-base @ file:///C:/b/abs_e4_l2p3q_7/croot/anaconda-cli-base_1741369468349/work
anaconda-client @ file:///C:/b/abs_0aa49y56xo/croot/anaconda-client_1743199592646/work
anaconda-navigator @ file:///C:/b/abs_5co9rfxmmi/croot/anaconda-navigator_1749741927270/work
anaconda-project @ file:///C:/b/abs_c3nd60wjs2

## FIM